In [1]:
%use krangl
val daten = DataFrame.readCSV("../daten/virusErde14.csv") //Liest die csv-Datei und wandelt sie in einen DataFrame um

<img src="../images/Lektion5.png" style="margin: 20px auto 20px 0px"/>
<h2 style="display:none">Lektion 5 - Verbindung von Lambda-Ausdrücken</h2>

In den meisten bis jetzt vorgestellten Beispielen wurde nur ein Lambda-Ausdruck verwendet. Jedoch können auch Mehrere hintereinander gestellt oder ineinander verschachtelt werden. Außerdem besteht die Möglichkeit, in den Funktionsrümpfen wiederum mit Methoden zu arbeiten. Es folgen einige Beispiele. Dabei wird empfohlen, den Code erst vollständig einzukommentieren und dann Anweisung für Anweisung durchzugehen.
#### Beispiel 1
Gesucht ist der amerikanische Bundesstaat mit den meisten bestätigten Fällen.

In [2]:
val bundesstaat = daten.rows.filter{ it["Country_Region"] == "US" } //Filtert nur die Einträge heraus, deren Country_Region US ist.
    .groupBy { it["Province_State"] }   //Gruppiert die Einträge nach dem Bundesstaat. Zurückgegeben wird eine Map mit dem Namen der Bundesstaaten als Schlüssel und einer Liste, die alle Zeilen des Bundesstaates umfasst. 
                                        //Jeder Eintrag in der Liste ist jedoch wieder eine Map, die als Schlüssel den Spaltennamen und als Wert den Eintrag an der jeweiligen Stelle besitzt.
    .mapValues { //Iteriert über die Map und setzt jeden Wert eines Paares auf die Rückgabe des Lambda-Ausdrucks
        it.value.sumBy { it["Confirmed"] as Int }   //it ist hier ein Map-Paar bestehend aus Bundesstaat und den zugehörigen Zeilen als Liste
                                                    //value greift auf die Liste des Bundesstaats zu
                                                    //sumBy() iteriert über jeden Eintrag der Liste und summiert den Wert, der dem Schlüssel Confirmed zugeordnet ist
    }                                               //Das Ergebnis ist eine Map, deren Schlüssel die Namen der Bundesstaaten und der zugehöige Werte die Summe an bestätigten Fällen sind
    .maxByOrNull { it.value } //Iteritert durch die Map an bestätigten Fällen jedes Bundesstaates und gibt das Paar zurück, dessen Fälle am höchsten sind
    ?.key //Greift auf den Wert des Paares zu. Es muss ein sicherer Zugriff verwendet werden, da maxByOrNull einen nullable Wert zurückgibt
                            
println("Das amerikanische Bundesland mit den meisten bestätigten Fällen ist: $bundesstaat.")
//Zum Nachvollziehen:
//bundesstaat

Das amerikanische Bundesland mit den meisten bestätigten Fällen ist: California.


#### Beispiel 2
Gesucht ist der Eintrag, dessen Koordinaten am nähesten an vorgegebene Längen- und Breitengraden sind. Die angegebenen Koordinaten entsprechen der Position der Universität Bayreuth.

In [3]:
val laengengrad = 49.92987060546875
val breitengrad = 11.586241722106934

fun Double.betrag() = if(this < 0) -this else this //gibt den Betrag eines Double-Werts zurück

val eintrag = daten.rows.filter { (it["Lat"] as String != "") && (it["Long_"] as String != "") } //Filtert alle Einträge mit leeren Koordinaten heraus  
        .minByOrNull { ((it["Lat"] as String).toDouble() - laengengrad).betrag() + ((it["Long_"] as String).toDouble() - breitengrad).betrag() } //Gibt den Eintrag zurück, dessen Differenz der Koordinaten zu den Angegebenen am kleinsten ist.

println("Der Eintrag, der am nähsten zu den gegebenen Koordinaten ist, besitzt die Werte:\n$eintrag")

Der Eintrag, der am nähsten zu den gegebenen Koordinaten ist, besitzt die Werte:
{FIPS=, Admin2=, Province_State=Bayern, Country_Region=Germany, Last_Update=2021-04-08 04:21:13, Lat=48.7904, Long_=11.4979, Confirmed=517336, Deaths=13419, Recovered=467100, Active=36817, Combined_Key=Bayern, Germany, Incident_Rate=3956.1599578365253, Case_Fatality_Ratio=2.5938654955386826}


#### Beispiel 3
In der Spalte *Incident_Rate* ist die Anzahl der bestätigten Fälle je 100.000 Einwohner zu finden. In diesem Beispiel sollen die 3 Länder mit der höchsten Durchseuchung in Prozent (Anteil der bestätigten Fälle an der Gesamtbevölkerung) ausgegeben werden. Wobei, falls eine Untergliederung in Regionen zu finden ist, der Mittelwert ausschlaggebend sein soll.

In [4]:
val rangliste = daten.rows.filter { it["Incident_Rate"] as String != "" } //Filtert alle Zeilen heraus, die keinen Eintrag in der Spalte Incident_Rate aufweisen
    .groupBy { it["Country_Region"] }   //Gruppiert die Einträge nach dem Land. Zurückgegeben wird eine Map mit dem Namen des Landes als Schlüssel und einer Liste, die alle Zeilen des Landes umfasst. 
                                        //Jeder Eintrag in der Liste ist jedoch wieder eine Map, die als Schlüssel den Spaltennamen und als Wert den Eintrag an der jeweiligen Stelle besitzt.
    .mapValues { //Iteriert über die Map und setzt jeden Wert eines Paares auf die Rückgabe des Lambda-Ausdrucks
        val inzidenzen = MutableList<Double>(0) { 0.0 } //Erstellt für jedes Land eine leere Liste, die mit den Inzidenzraten gefüllt werden soll
        it.value.forEach{ //Iteriert über alle Einträge jedes Land
            inzidenzen.add((it["Incident_Rate"] as String).toDouble()) //Fügt der Liste die Inzidenzrate der Region hinzu
        }  
        (inzidenzen.average() / 100000) * 100 //Berechnet den Durchschnitt der Wert der Liste, wandelt ihn in Prozent um und gibt ihn zurück
    }
    .toList() //Konvertiert die Map in eine Liste aus Paaren, welche aus dem Schlüssel und dem berechneten Wert besteht, da eine Map nicht sortiert werden kann
    .sortedByDescending { (schluessel, wert) -> wert} //Sortiert die Liste nach den Werten absteigend
    .take(3) //Löscht alle Einträge abgesehen von den ersten Drei
    
//Ausgabe
for (i in 0 until rangliste.size){
    println("Platz ${i+1}: ${rangliste[i].first} mit einer Durchseuchung von ${rangliste[i].second} %")
}
//Zum Nachvollziehen:
//rangliste

Platz 1: Andorra mit einer Durchseuchung von 16.00077654824306 %
Platz 2: Montenegro mit einer Durchseuchung von 14.856654279354586 %
Platz 3: Czechia mit einer Durchseuchung von 14.574466555271082 %
